<a href="https://colab.research.google.com/github/johndelara1/AIRBNB-cognitivo/blob/master/TREINAMENTO_SPACY_GCP_MARCAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proposta do SPACY?
- Com o RE conseguimos identificar bem as palavras que temos mapeadas, mas não conseguimos mapear duas situações: marcas que não estão na nossa lista de "MarcaPalavras" (novas marcas) e nem casos onde não temos sinonimos da marca na descrição

- Proposta: utilizar o Spacy para identificarmos essas duas situações que não são mapeadas pelo RE (novas marcas) e marcas sem a palavra "Marca" antes do nome da marca

In [ ]:
#!pip install spacy-pytorch-transformers[cuda100]==0.2.0
# !pip install spacy_sentiws
#!pip install spacy==2.3.2
#!python -m spacy download pt_core_news_lg
#!python -m spacy link pt_core_news_lg pt_core_news_lg
# !pip install -U spacy[cuda100]
# pip install cupy-cuda100
from google.colab import drive
drive.mount('/content/gdrive')
import sys
sys.path.append('/content/gdrive/My Drive')
import pandas as pd
import re
import funcoes
import spacy
import glob
spacy.require_gpu()
import pandas as pd
from spacy import displacy
import random
# !pip install google-cloud-automl
# from google.cloud import automl
# nlp = spacy.load("/content/gdrive/MyDrive/train_10000_rows/")

Mounted at /content/gdrive
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
bibliotecas carregadas!


# Testar o modelo

In [ ]:
'ec b aparelho conversor protocolos tipo gateway interconexao redes comunicacao marca lonwork bacnet utilizadas respectivamente sistemas alarme protecao contra incendios sistemas gestao predial bms referenc'
# OK para palavras depois de marca
'pcs software maintenance station gx yb oq cdp icss software upgrade engineering v floating license f siemens user swcomponent djf system supervisory paraty serial,1'
# ERRO pois achou 'maintenance'
# # ERRO pois achou 'cdp'
'es tg ab siemens s unidade processamento central plc tecnologia memoria trabalho kbyte integrada conector dianteiro tecnologico x pin cartao memoria micro min mb necessarios,1'
# OK siemens
'fe bb roteador digital nokia marca isam capacidade conexao fio aplicado redes opticas marca nokia ref g w h plug brazil'
# trouxe previsões mais voltados a marca 'nokia' verificamos mais exemplos no treinamento
# trouxe a marca isam no segundo modelo
'marca  consul câmera segurança marca intelbras ic ns s ug j'
# Boa no novo modelo

'marca  consul câmera segurança marca intelbras ic ns s ug j'

In [ ]:
test_text = 'fe bb roteador digital nokia marca isam capacidade conexao fio aplicado redes opticas marca nokia ref g w h plug brazil'
doc = nlp(test_text)
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)


isam 35 39 category


#  Treinar Modelo

In [ ]:
def uploadCSV(path):
    df = pd.read_csv(path, encoding='latin_1', sep=';')
    return df
def train_spacy(data, iterations):
    TRAIN_DATA = data
    nlp = spacy.blank('pt')  # create blank Language class
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)

    # add labels
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get('entities'):
            ner.add_label(ent[2])
    
    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training(device=-1)#device=-1 ou vazio -> https://support.prodi.gy/t/will-a-gpu-make-training-faster/187/3
        for itn in range(iterations):
            print("Statring iteration " + str(itn))
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                    [text],  # batch of texts
                    [annotations],  # batch of annotations
                    drop=0.2,  # dropout - make it harder to memorise data
                    sgd=optimizer,  # callable to update weights
                    losses=losses)
            print(losses)
    return nlp

def update_train_spacy(data, iterations):
    TRAIN_DATA = data
    nlp = spacy.load("/content/gdrive/MyDrive/train_10000_rows/")
    
    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.resume_training(device=-1)
        for itn in range(iterations):
            print("Statring iteration " + str(itn))
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                    [text],  # batch of texts
                    [annotations],  # batch of annotations
                    drop=0.2,  # dropout - make it harder to memorise data
                    sgd=optimizer,  # callable to update weights
                    losses=losses)
            print(losses)
    return nlp

In [ ]:
# TRAIN_DATA = list()
# for arquivo in range(0, len(files)):
#     df = uploadCSV(files[arquivo])
#     # df = df.drop_duplicates(subset=['text']).reset_index()
#     # df.text = df.test.map(lambda x: x.lower())
#     for i in range(0,len(df)):
#         if pd.notna(df['start'][i]) and pd.notna(df['finish'][i]):
#             # print(i, '|' * 50)
#             # print(df['Descricao'][i], "\n", df['Marca'][i], '=>', df['Marca Posição inicial'][i],  '|', df['Marca Posição Final'][i], '\n', df['Modelo'][i], '=>', df['Modelo Posição inicial'][i],  '|', df['Modelo Posição Final'][i])
#             # texto = df['Descricao'][i]
#             # print(texto[int(df['Marca Posição inicial'][i]):int(df['Marca Posição Final'][i])])
#             # print(texto[int(df['Modelo Posição inicial'][i]):int(df['Modelo Posição Final'][i])])
#             # print('-'* 50)
#             # print(df['Descricao'][i], int(df['Marca Posição inicial'][i]), int(df['Marca Posição Final'][i]))
#             TRAIN_DATA.append((df['text'][i], {'entities': [(int(df['Marca Posição inicial'][i]), int(df['Marca Posição Final'][i]), 'MARCA')]}))
#             # TRAIN_DATA = list(set(TRAIN_DATA))

In [ ]:
# files = glob.glob("/content/drive/MyDrive/Bases Para Dicionário/*.csv")
%%time
df = funcoes.simple.select_dw("SELECT * FROM `simplediscoverydata.consumerZone.DS_GERAL_S_70000-MarcasParaTreinamentoSpacy` WHERE id_treinoETeste <= 20000")
TRAIN_SPACY = list()
for i in range(0,len(df)):
  if pd.notna(df['start'][i]) and pd.notna(df['finish'][i]):
    TRAIN_SPACY.append((df['text'][i], {'entities': [(int(df['start'][i]), int(df['finish'][i]), 'category')]}))

CPU times: user 1.12 s, sys: 35.2 ms, total: 1.15 s
Wall time: 4.25 s


In [ ]:
%%time
# Treinamento marca
prdnlp = update_train_spacy(TRAIN_SPACY, 20)

# Save our trained Model
modelfile = '/content/gdrive/MyDrive/train_20000_rows_/'
prdnlp.to_disk(modelfile)

Statring iteration 0


In [ ]:
# nlp = spacy.load("/content/gdrive/MyDrive/marcas")
# nlp = spacy.load("marcas_intelbras")
# nlp = spacy.load("train_1000_rows")
# nlp = spacy.load("/content/gdrive/MyDrive/train_10000_rows/")

# Tratativas de texto

In [ ]:
def remove_stopwords_fast(text):
    doc = nlp(text.lower())
    result = [token.text for token in doc if token.text not in nlp.Defaults.stop_words]
    return " ".join(result)
def remove_stopwords(text):
    doc = nlp(text.lower()) #1
    result = [] #2
    for token in doc: #3
        if token.text in nlp.Defaults.stop_words: #4
            continue
        result.append(token.text)#5
    return " ".join(result) #6
def remove_pronoun(text):
    doc = nlp(text.lower())
    result = [token for token in doc if token.lemma_ != '-PRON-']
    return " ".join(result)


def process_text(text):
    doc = nlp(text.lower())
    result = []
    for token in doc:
        if token.text in nlp.Defaults.stop_words:
            continue
        if token.is_punct:
            continue
        if token.lemma_ == '-PRON-':
            continue
        result.append(token.lemma_)
    return " ".join(result)

def calculate_similarity(text1, text2):
    base = nlp(process_text(text1))
    compare = nlp(process_text(text2))
    return base.similarity(compare)

# GCP - Para treinarno GCP extração de variáveis

In [ ]:
def create_dataset(project_id, display_name):
    """Create a dataset."""
    # [START automl_language_entity_extraction_create_dataset]
    from google.cloud import automl

    # TODO(developer): Uncomment and set the following variables
    # project_id = "YOUR_PROJECT_ID"
    # display_name = "YOUR_DATASET_NAME"

    client = automl.AutoMlClient()

    # A resource that represents Google Cloud Platform location.
    project_location = f"projects/{project_id}/locations/us-central1"
    metadata = automl.TextExtractionDatasetMetadata()
    dataset = automl.Dataset(
        display_name=display_name, text_extraction_dataset_metadata=metadata
    )

    # Create a dataset with the dataset metadata in the region.
    response = client.create_dataset(parent=project_location, dataset=dataset)

    created_dataset = response.result()

    # Display the dataset information
    print("Dataset name: {}".format(created_dataset.name))
    print("Dataset id: {}".format(created_dataset.name.split("/")[-1]))
    # [END automl_language_entity_extraction_create_dataset]

def create_model(project_id, dataset_id, display_name):
    """Create a model."""
    # [START automl_language_entity_extraction_create_model]
    from google.cloud import automl

    # TODO(developer): Uncomment and set the following variables
    # project_id = "YOUR_PROJECT_ID"
    # dataset_id = "YOUR_DATASET_ID"
    # display_name = "YOUR_MODEL_NAME"

    client = automl.AutoMlClient()

    # A resource that represents Google Cloud Platform location.
    project_location = f"projects/{project_id}/locations/us-central1"
    # Leave model unset to use the default base model provided by Google
    metadata = automl.TextExtractionModelMetadata()
    model = automl.Model(
        display_name=display_name,
        dataset_id=dataset_id,
        text_extraction_model_metadata=metadata,
    )

    # Create a model with the model metadata in the region.
    response = client.create_model(parent=project_location, model=model)

    print("Training operation name: {}".format(response.operation.name))
    print("Training started...")
    # [END automl_language_entity_extraction_create_model]

In [ ]:
# import os
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/content/api_google.json"
# create_dataset(project_id='simplediscoverydata', display_name='simpleDicoveryData')

In [ ]:
# create_model(project_id='simplediscoverydata', dataset_id='TEN3449098157344948224', display_name='simpleDicoveryData')

In [ ]:
# jsonl = df.to_json(orient='records', lines=True, force_ascii=False)
# # !pip install jsonlines
# import jsonlines
# g = open('data.jsonl', 'w')
# with g as outfile:
#   json.dump(jsonl, outfile, ensure_ascii=False)